# Introduction: `joblib` tutorial

In this tutorial, I will walk you through `joblib`, which is a package that provides
- an fast/easy/robust way to parallelize computations using `joblib.Parallel`
- computation caching using `joblib.Memory`

# Distributing code on a single machine using `joblib.Parallel`

`joblib` is a pure-python package that allows you to run `Python` code in parallel.

Scientific numerical experiments often require running the **same function**, **multiple times** using a different set of parameters each time (well known examples include cross-validation, or hyper-parameter selection).

### A simple example: executing multiple function calls in serial mode

Let's start with a very simple example: I want to run `my_fuction` multiple times for different sets of arguments:

In [ ]:
import time


# Here is the function that I want to parallelize -- a more realistic
# example would be named ``fit_model`` for instance.
def my_function(i):
    
    # Simulate a long-running computation. A real-life code could be np.linalg.svd(...).
    time.sleep(0.5)
    
    return i


# The set of arguments I want to run my function on. Each item in this list could
# be the value of a regularization parameter
my_args = list(range(10))

In [ ]:
%%time
results = [my_function(arg) for arg in my_args]
print(results)

The cell above took ~ 5 seconds to run -- it ran all of theese function calls one after the other,
which can be suboptimal when a computer has multiple processing units (CPUs), which is pretty much the case for all modern computers.

### Executing multiple function calls in parallel using `joblib`

In [ ]:
from joblib import Parallel, delayed

In [ ]:
%%time
results = Parallel(n_jobs=2)(delayed(my_function)(arg) for arg in my_args)
print(results)

The **exact** same code runs now on ~ 2.5 seconds, which is ~ half the time it took in the serial situation!

Advanced note: in standard CPU architectures, code in parallel is done using either **threads** or **processes**
By default, `joblib` relies on **processes**-based parallelism, because **thread**-based parallelism has limitations in `Python`. You can specify which kind of parallelism you want `joblib` to use using the `backend` option of the `Parallel` constructor:

In [ ]:
%%time
results = Parallel(n_jobs=2, backend='threading')(delayed(my_function)(arg) for arg in my_args)
print(results)

## Real life computations - Discussion, Pitfalls, Limits

The benchmark above is helpful to quickly demonstrate what `joblib` can do, but is not representative of a realistic situation because `my_function` does not perform any actual computation. For real-life use cases, using simple personal computers, the speed up that `joblib` will provide is dependent on your code's computational specificities. Here are a few remarks:

#### Competing with lower-level parallelization

Python is a language that can execute arbitrary C code, which is generally much faster to run than Python code. For instance, most of `numpy` is written in C. When possible, `numpy` parallelizes matrix operations at a low level using linear algebra libraries such as `OpenBLAS` or `MKL`. This **low-level** parallelization competes with the **higher-level** parallelization that what joblib relies on. Using both parallelization level at the same time can lead to **over-subscription** (too many instructions threads executed concurrenltly), which can **decrease** your code performance, especially on machines with manu `CPU`s.


Luckly, both parallelism can be enabled, disabled using methods that I will discuss below. Here is how switchin on/off each parallelism performs on a standard numpy workload:

![title](benchmarks/results.png)

(Note: hese benchmarks are run on a 24 physical/48 virtual CPU machine. In the `inner` case, the BLAS library is free to use any number of threads. In the `outer` case, `joblib` spawns 48 workers (which is un-necessary when the number of sdv run is below 48, but I decided to keep the resource logic simple), and I specifically ask the BLAS library to use only 1 thread.

As you can see, when a few outer tasks (svd) are computed, inner-parallelism takes advantage of the 48 CPUs while `joblib` cannot. But as the number of tasks increases, the inner method scales less well than the joblib one.

Keep in mind that this scenario in a close to worst-case scenario in `joblib` as there is a lot of complexity happening outdside of the `Python` interpreter (most of the work is done by low-level `C` routines "outside" of Python, ask me later if you want to know what that means).

Here is the function I ran, and a small benchmark on my machine.

In [ ]:
import numpy as np


def run_svd(random_seed):
    rs = np.random.RandomState(random_seed)
    array = rs.randn(1500, 1500)
    U, S, V = np.linalg.svd(array)
    return S[0]

In [ ]:
%%time
results = [run_svd(seed) for seed in range(4)]
print(results)

In [ ]:
%%time
results = Parallel(n_jobs=2)(delayed(run_svd)(seed) for seed in range(4))
print(results)

#### How to disable inner parallelism:

When using `joblib`, it is recommended to disable lower-level serialization, by defining/exporting the following environment variables:
```bash
OPENBLAS_NUM_THREADS=1  # if numpy uses openblas
MKL_NUM_THREAD=1        # if numpy uses MKL. Will be the case for a numpy coming from `conda`
```

You can either put these lines in your favorite shell configuration file (`~/.bashrc`/`~/.bash_profile`/`~/.profile`...)
```bash
export OPENBLAS_NUM_THREADS=1  # if numpy uses openblas
export MKL_NUM_THREAD=1        # if numpy uses MKL. Will be the case for a numpy coming from `conda`
```

Or use them in a case by case basis:
```
OPENBLAS_NUM_THREADS=1  MKL_NUM_THREAD=1 jupyter-notebook  # start a jupyter-notebook in which blas parallelization is disabled
```


**Advanced note**
Enabling/disabling blas can now be done programatically for each chunk of code using our new `threadpoolctl` package. This is in early development, and only recommended for "experts".


Advanced note: `numpy` functions generally release the Python Global Interpreter Lock (GIL), which means that
thread-based parallelization is likely to be as efficient as process-based parallelization:

In [ ]:
%%time
results = Parallel(backend='threading', n_jobs=2)(delayed(run_svd)(seed) for seed in range(4))
print(results)

#### A better use-case for `joblib`: Python-level iterative algorithms
Not all Python code is efficient numpy code - iterative algorithms such as gradient descent or MCMC methods generally use a Python `for`-loop. For such setups, `joblib` will be more likely to yield higher speedups:

In [ ]:
def run_mcmc(seed):
    rs = np.random.RandomState(seed)
    n_iter = 1000000
    theta = 0
    for i in range(n_iter):
        theta += rs.randn(10)
    return theta

In [ ]:
%%time
results = [run_mcmc(i) for i in range(4)]

In [ ]:
%%time
results = Parallel(n_jobs=2)(delayed(run_mcmc)(i) for i in range(4))

Advanced note: in situation involving iterative logic, another way to improve the execution performance is to use rely just-in-time compilation. A few pointers: `jax`, `numba` (and even `pypy!`)

### Issues preventing ideal speedups

In an ideal setting, One would typically expect that `joblib` yields a linear speedup (in log-log scale) w.r.t the number of available CPUs. That is:

$$\text{running_time(n_cpu)} = \frac{\text{serial_running_time}}{\text{n_cpu}}$$

In practice, this is not the case, because of various low-level phenomenas:

#### Resources sharing


Modern CPUs speed-up data access using caches. Some of these caches are shared between CPUs. When running code in parallel, multiple CPUs will compete to use the same shared cache, which tends to slow things down.

#### Interprocess communication

process-based parallelism is usually the recommended parallelism form in Python as thread-based parallelism can be compromised by the GIL. However, processes do not share memory, so child processes must be transmitted data (the function, the inputs...) from the original process. Serializing data is done in serial. It is a fixed time addition whose importance w.r.t the total running time increases as the number of CPUs increases.


![title](benchmarks/results_2.png)

# Automatic caching using `joblib.Memory`


Among the main pain points of numerical experiments, there is one that stands out: having a long-running experiment erroring out during its execution. Usually, intermediate computations will be lost, which can mean days of computing and human work wasted.

`joblib` provides a way to cache a function:

In [ ]:
from joblib import Memory
memory = Memory('joblib-cache-directory')

# caching functions defined inside notebooks is tricky. For maximum
# robustness, my_function should be moved to a python file when using
# joblib.Memory
from my_module import my_function
my_function_cached = memory.cache(my_function)

In [ ]:
%%time

results = Parallel(n_jobs=2)(delayed(my_function_cached)(arg) for arg in my_args)
print(results)

In [ ]:
%%time
results = Parallel(n_jobs=2)(delayed(my_function_cached)(arg) for arg in my_args)
print(results)

Notice the drop in execution time between the two cells above? In the first cell, my_function did not have
any cached results yet; executing those function calls took the same time as in the not-cached case.

But in the next cells, the exact same `Parallel` call took only 10ms: `joblib` did not re-execute all of these function calls, but only loaded the already-computed results!

**Important remarks**: each time the source codes of a `joblib.Memory`-cached function changes, the cached associated to this function is cleared.

#### From single-machine parallelism to multi-machine parallelism

- Usually, `joblib` can yield improvements proportional to the number of CPUs of the machine you are using. If your machine has 4 CPUs, you can expect the total running time of your function calls to be divided by up to 4 as compared to the serial case. Usually, laptops have from 2 up to 12 CPUs.

- `joblib` provides a way to quickly parallelize code on a single machine. But scientific institutions such as our often have access to a larger set of computational resources, such as a slurm cluster. Running code in various nodes of a slurm cluster in parallel can be done in various ways (submitting `batch` scripts for instance). But doing this is error prone, and not the areas of expertise of researchers. 